In [2]:
import re
import time
from typing import Iterator
import requests
import lxml
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [242]:
#クローラー
def main():
    """
    クローラーのメインの処理。
    """
    client = MongoClient('localhost', 27017)  # ローカルホストのMongoDBに接続する。
    collection = client.scraping.coe  # scrapingデータベースのcoeコレクションを得る。
    # データを一意に識別するキーを格納するkeyフィールドにユニークなインデックスを作成する。
    collection.create_index('key', unique=False)
    
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'

    session = requests.Session()
    response = session.get('https://allianceforcoffeeexcellence.org/competition-auction-results/')  # 一覧ページを取得する。
    urls = scrape_list_page(response)  # 詳細ページのURL一覧を得る。
    for url in urls:
        key = extract_key(url)  # URLからキーを取得する。
        coe = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
        if not coe:  # MongoDBに存在しない場合だけ、詳細ページをクロールする。
            time.sleep(3)
            response = session.get(url)  # 詳細ページを取得する。
            try:
                results = scrape_detail_page(response)
                for result in results:
                    result = {k.replace('.',''): v for k, v in result.items() if type(k) != int}
                    collection.insert_one(result)  # 表の情報をMongoDBのcoeコレクションに保存する。
            #No tableの場合
            except ValueError:
                continue        

#         print(data)  # 表の情報を表示する。

#スクレイピング
def scrape_list_page(response: requests.Response) -> Iterator[str]:
    """
    一覧ページのResponseから詳細ページのURLを抜き出すジェネレーター関数。
    """
    html = lxml.html.fromstring(response.text)
    html.make_links_absolute(response.url)

    for a in html.cssselect('#menu-coe-country-programs-menu a[href^="https://"]'):
        #:not(#menu-item-65034)
        url = a.get('href')
        yield url


def scrape_detail_page(response: requests.Response) -> dict:
    """
    詳細ページのResponseから表の情報をdictで取得する。
    """
#     html = lxml.html.fromstring(response.text)
    tables = pd.read_html(response.content)
    data = None
    #列名修正ver
    if 'rank' in list(table[0].iloc[0].str.lower().values):
        tables[0].columns = tables[0].iloc[0]
        tables[0].drop(index=0, inplace=True)
        data = tables[0]
        for x in range(1, len(tables)):
            if len(tables[0])-3 <= len(tables[x]) <= len(tables[0])+3:
                tables[x].columns = tables[x].iloc[0]
                tables[x].drop(index=0, inplace=True)
                data = pd.concat([data, tables[x]], axis=1, sort=False)
        #オークションに関するカップ情報の辞書を作成
        data = data.to_dict(orient='records')
        #key, urlの追加
        for i in range(len(data)):
            data[i]['key'] = extract_key(response.url)
            data[i]['url'] = response.url
        return data  # dictを返す。
    
    else:
        #オークション情報の整形
        data = tables[0]
        #目的となるtableのみdataに追加
        for x in range(1, len(tables)):
            if len(tables[0])-1 <= len(tables[x]) <= len(tables[0])+1:
                data = pd.concat([data, tables[x]], axis=1, sort=False)
        #オークションに関するカップ情報の辞書を作成
        data = data.to_dict(orient='records')
        #key, urlの追加
        for i in range(len(data)):
            data[i]['key'] = extract_key(response.url)
            data[i]['url'] = response.url
        return data  # dictを返す。
    

def extract_key(url: str) -> str:
    """
    URLからキー（URLの末尾のISBN）を抜き出す。
    """
    m = re.search(r'/([^/]+)/$', url)  # 最後の/から文字列末尾までを正規表現で取得。
    return m.group(1)

if __name__ == '__main__':
    main()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:81: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [239]:
#動作確認
url = requests.get('https://allianceforcoffeeexcellence.org/brazil-2019/')
# url = requests.get('https://allianceforcoffeeexcellence.org/bolivia-2009/')
table = pd.read_html(url.content)#多重リスト
if 'rank' in list(table[0].iloc[0].str.lower().values):
    table[0].columns = table[0].iloc[0]
    table[0].drop(index=0, inplace=True)
    data = table[0]
    for x in range(1, len(table)):
        if len(table[0])-2 <= len(table[x]) <= len(table[0])+2:
            table[x].columns = table[x].iloc[0]
            table[x].drop(index=0, inplace=True)
            data = pd.concat([data, table[x]], axis=1, sort=False)
    data = data.to_dict(orient='records')
else:
    data = table[0]
    for x in range(1, len(table)):
        if len(table[0])-2 <= len(table[x]) <= len(table[0])+2:
            data = pd.concat([data, table[x]], axis=1, sort=False)
    data = data.to_dict(orient='records')
data

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  del sys.path[0]


[{'RANK': '1a',
  'WEIGHT (kg)': '180',
  'FARM': 'Fazenda Paie Filho',
  'FARMER': 'André Luis Aguila Ribeiro',
  'REGION': 'Sul de Minas',
  'SCORE': '92.23',
  'VARIETY': 'Yellow Catuaí',
  'PROCESS': 'Natural',
  'ROLE': 'Head Judge',
  'NAME': 'Eduardo Ambrocio',
  'COMPANY': 'Eduardo Ambrocio',
  'COUNTRY': nan,
  'Rank': '1a',
  'Farm': 'Fazenda Paie Filho',
  'Weight (lbs.)': '396.84',
  'High Bid': '$60.10/lb',
  'Total Value': '$23,850.08',
  'Company Name': 'Maruyama Coffee, Goodboybob Coffee, Harrods',
  'Commissions': '$5,645.05'},
 {'RANK': '1b',
  'WEIGHT (kg)': '180',
  'FARM': 'Fazenda Paie Filho',
  'FARMER': 'André Luis Aguila Ribeiro',
  'REGION': 'Sul de Minas',
  'SCORE': '92.23',
  'VARIETY': 'Yellow Catuaí',
  'PROCESS': 'Natural',
  'ROLE': 'ACE Rep',
  'NAME': 'Darrin Daniel',
  'COMPANY': 'Darrin Daniel',
  'COUNTRY': nan,
  'Rank': '1b',
  'Farm': 'Fazenda Paie Filho',
  'Weight (lbs.)': '396.84',
  'High Bid': '$50.80/lb',
  'Total Value': '$20,159.47',
  '

In [6]:
import pandas as pd
url = requests.get('https://cupofexcellence.org/directory/90-4/')
table = pd.read_html(url.content)#多重リスト
data = {}
data.update(zip(table[-1][0], table[-1][1]))
data

{'Rank': '1b',
 'Farm Name': 'Esperanza',
 'Farmer/Rep.': 'Hilda Leguia Gonzales',
 'Score': '90',
 'Altitude': '1848',
 'Year': '2020',
 'City': 'La Convencion',
 'Region': 'Cusco',
 'Program': 'Perú 2020',
 'Auction': 'Cup of Excellence',
 'Overall': 'Complex, Creamy, Floral, High acidity, Lingering, Peach jam, Round, Silky, Thickness',
 'Aroma/Flavor': 'Floral (7), Grape (3), Honey (3), Juicy (3), Chocolate (2), Creamy (2), Jasmine (2), Lemon & Lime (2), Lime (2), Mango (2), Melon (2), Milk Chocolate (2), Muscat (2), Orange (2), Strawberry (2), Apple, Apricot, Black Cherry, Black Currant, Black Tea, Blackberry, Butter, Citrus, Cocoa, Cream, Fruity, Grassy, Hibiscus, Jammy, Lemongrass, Liquorice, Lychee, Mandarin Orange, Molasses, Peach, Pear, Pomelo, Powdery, Red Currant, Roasted Hazelnut, Rose Hips, Salted lemon, Savory, Stone Fruit, Sweet & Sugary, Tomato, Tropical Fruit, WatermelonCitric Acid (4), Tartaric Acid (2), Bright, Complex, Lively, Malic Acid, Pungent, Refine, Sweet, Tar